This notebook has custom AlexNet. It measures:


1.   Sparsity of weights(one-time)
2.   Layerwise CONV layer activation sparsities
3.   Accuracy of the model
4. Layerwise #MAC ops

# Imports

In [1]:
import numpy as np
import torch
import time
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print(device)

cpu


In [2]:
#Loading the dataset and preprocessing

batch_size = 1

train_dataset = torchvision.datasets.MNIST(root = './data',
                                           train = True,
                                           transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                           download = True)


test_dataset = torchvision.datasets.MNIST(root = './data',
                                          train = False,
                                          transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1325,), std = (0.3105,))]),
                                          download=True)


#                                                   transforms.Normalize(mean = (0.1325,), std = (0.3105,))

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


# Model

**Custom conv2d function**

In [5]:
def myconv2d(input, weight, bias=None, stride=(1,1), padding=(0,0), dilation=(1,1), groups=1):
    """
    Function to process an input with a standard convolution
    """
    mul_count = 0
#     print('input', input.shape)
#     print('wt', weight.shape)
    batch_size, in_channels, in_h, in_w = input.shape
    out_channels, in_channels, kh, kw = weight.shape
    out_h = int((in_h - kh + 2 * padding[0]) / stride[0] + 1)
    out_w = int((in_w - kw + 2 * padding[1]) / stride[1] + 1)
    unfold = torch.nn.Unfold(kernel_size=(kh, kw), dilation=dilation, padding=padding, stride=stride)
    inp_unf = unfold(input)
    w_ = weight.view(weight.size(0), -1).t()
    if bias is None:
        out_unf = inp_unf.transpose(1, 2).matmul(w_).transpose(1, 2)
        mul_count += batch_size*out_channels*out_h*out_w*in_channels*kh*kw
    else:
        out_unf = (inp_unf.transpose(1, 2).matmul(w_) + bias).transpose(1, 2)
        mul_count += batch_size*out_channels*out_h*out_w*in_channels*kh*kw
    out = out_unf.view(batch_size, out_channels, out_h, out_w)
#     print(out)
    return (out.float(), mul_count)
    # return out.float()

##############################################################################################

class comp_vector():
  def __init__(self, arr):
    self.x = arr.size(dim=2)
    self.y = arr.size(dim=1)
    self.c = arr.size(dim=0)
    self.index_vector = []
    self.data_vector = []
    for i in range(self.c):
      # print(arr[i])
      self.index_vector.append(np.flatnonzero(arr[i].cpu()))
      self.data_vector.append(arr[i].ravel()[self.index_vector[-1]])

    # index_vector = np.flatnonzero(arr)
    # data_vector = arr.ravel()[index_vector]

  def get_index_vector(self):
    return self.index_vector

  def get_data_vector(self):
    return self.data_vector


def conv_compressed(comp_inp, comp_wt, stride=1):
#     print('called conv_compressed')
    acc_x, acc_y, acc_c = int((comp_inp.x - comp_wt.x)//stride  + 1) , int((comp_inp.y - comp_wt.y)//stride  +1), comp_wt.c
#     print(acc_x, acc_y, acc_c)
    mult_count = 0
    # print(acc_x, acc_y, acc_c)
    acc_buf = torch.FloatTensor(acc_x, acc_y).zero_()
    inp_index_vector = comp_inp.get_index_vector()
    inp_data_vector = comp_inp.get_data_vector()
    wt_index_vector = comp_wt.get_index_vector()
    wt_data_vector = comp_wt.get_data_vector()
    # print(inp_index_vector[0])
    # print(len(inp_index_vector[0]))
    for c in range(acc_c):
      for i in range(len(inp_index_vector[c])):
        for j in range(len(wt_index_vector[c])):
          inp_x = inp_index_vector[c][i]//comp_inp.x
          inp_y = inp_index_vector[c][i]%comp_inp.y
          wt_x = wt_index_vector[c][j]//comp_wt.x
          wt_y = wt_index_vector[c][j]%comp_wt.y

          out_x = (inp_x - wt_x)
          out_y = (inp_y- wt_y)
          if out_x%stride==0 and out_y%stride==0:
            out_x = out_x//stride
            out_y = out_y//stride
            # print(out_x, out_y,c,i,j,)
            if 0<=out_x<acc_x and 0<=out_y<acc_y:
              # print("yes")
              acc_buf[out_x][out_y]+=float(inp_data_vector[c][i] * wt_data_vector[c][j])
              mult_count +=1
    
    return (acc_buf,mult_count)

def myconv2d_sparse(input, weight, bias=None, stride=(1,1), padding=(0,0), dilation=(1,1), groups=1):
  input = torch.nn.functional.pad(input, (padding[1], padding[1], padding[0], padding[0]), "constant", 0)
#   print(input.size())
  comp_in = comp_vector(input[0])
  in_x = input.size(dim=3)
  in_y = input.size(dim=2)
  wt_x = weight.size(dim=3)
  wt_y = weight.size(dim=2)
  c = weight.size(dim=1)
  k = weight.size(dim=0)
  out = torch.empty(size=(1,k, int((in_x-wt_x)/stride[0]+1), int((in_y-wt_y)/stride[1]+1)))

  mult_count = 0
  for i in range(k):
    comp_wt = comp_vector(weight[i])
    out[0][i], num =conv_compressed(comp_in, comp_wt, stride[0])
    out[0][i] += bias[i]
    mult_count+=num
#   print(out)
  return (out,mult_count)

######################################################################################################

# ALGO 3
def compute_weight_list(kernel):    
    kernels = []
    filter_count = kernel.shape[0]
    depth = kernel.shape[1]
    height = kernel.shape[2]
    width = kernel.shape[3]
    for f in range(filter_count):
        weight_list = []
        for k in range(depth):
            for i in range(height):
                for j in range(width):
                    w = kernel[f][k][i][j]
                    if w < 0:
                        weight_list.append(tuple((w, k, i, j)))
        sorted_weight_list = sorted(weight_list, key = lambda x: x[0])
        kernels.append(sorted_weight_list)
    return kernels

def compute_conv_onlypred(img, weight_list, weights, r, c, bias=0):
    img_out_cell = 0
    conv_mult_count = 0
    depth = weights.shape[0]
    height = weights.shape[1]
    width = weights.shape[2]
    for k in range(depth):
        for i in range(width):
            for j in range(height):          
                if weights[k][i][j] > 0:
                    conv_mult_count += 1 
                    img_out_cell += img[0][k][r+i][c+j]*weights[k][i][j]

    img_out_cell+=bias
    
    for tup in weight_list:
        conv_mult_count += 1
        img_out_cell += tup[0]*img[0][tup[1]][r+tup[2]][c+tup[3]]
        if img_out_cell < 0:
            break
    return img_out_cell, conv_mult_count

def compute_filter_conv_onlypred(img, weight_list, weights, kernel_id,stride=(1,1), padding=(0,0), bias=0):
    width_out = int((img.shape[3]+2*padding[1]-weights.shape[2])/stride[1]+1)
    height_out = int((img.shape[2]+2*padding[0]-weights.shape[1])/stride[0]+1)
    img_out_channel = torch.zeros(width_out,height_out)
    filter_mult_count = 0
    # print(img.shape[2]+2*padding[0]-weights.shape[1], img.shape[3]+2*padding[1]-weights.shape[2])
    for r in range(0,img.shape[2]+2*padding[0]-weights.shape[1]+1,stride[0]):
        for c in range(0,img.shape[3]+2*padding[1]-weights.shape[2]+1,stride[1]):
            r_out = int(r/stride[0])
            c_out = int(c/stride[1])
            # print(r_out, c_out)
            img_out_channel[r_out][c_out], mult_count = compute_conv_onlypred(img, weight_list, weights, r, c, bias)
            # img_out_channel[r_out][c_out] += bias
            filter_mult_count += mult_count
    return img_out_channel, filter_mult_count

def myconv2d_onlypred(img, weights, bias=None, stride=(1,1), padding=(0,0), dilation=(1,1), groups=1):
    layer_mult_count = 0
    filter_count = weights.shape[0]
    depth = weights.shape[1]
    height = weights.shape[2]
    width = weights.shape[3]
    channels_out=filter_count
    width_out = int((img.shape[3]+2*padding[1]-width)/stride[1]+1)
    height_out = int((img.shape[2]+2*padding[0]-height)/stride[0]+1)
    img_conv_output = torch.zeros(1, channels_out, width_out, height_out)
    filters_list = compute_weight_list(weights)
    for kernel_id in range(filter_count):
        if kernel_id%8==0:
            print("kernel_id", kernel_id)
        weight_list = filters_list[kernel_id]
        img_conv_channel, mult_count = compute_filter_conv_onlypred(img, weight_list, weights[kernel_id], kernel_id, stride, padding, bias[kernel_id])
        img_conv_output[0][kernel_id] = img_conv_channel
        layer_mult_count += mult_count
    return (img_conv_output, layer_mult_count)


######################################################################################


# ALGO 4
class comp_vector_pred():
  def __init__(self, arr):
    self.y = arr.size(dim=2)
    self.x = arr.size(dim=1)
    self.c = arr.size(dim=0)
    self.pos_vector = [] #stores tuples of (data, index)
    self.neg_vector = []

    for k in range(self.c):
            for i in range(self.x):
                for j in range(self.y):
                    w = arr[k][i][j]
                    if w > 0:
                      self.pos_vector.append(tuple((w, k, i, j)))
                    elif w<0:
                      self.neg_vector.append(tuple((w, k, i, j)))

    self.neg_vector = sorted(self.neg_vector, key = lambda x: x[0])

  def get_pos_vector(self):
    return self.pos_vector

  def get_neg_vector(self):
    return self.neg_vector


def compute_conv_sparsepred(input, weight, comp_wt, r, c, bias=0):
  img_out_cell = 0
  conv_mult_count = 0
  pos = comp_wt.get_pos_vector()
  neg = comp_wt.get_neg_vector()

  x = weight.shape[1]
  y = weight.shape[2]
  k = weight.shape[0]

  mult_nonzero = 0
  for channel in range(k):
    inp_window = input[0][channel][r:r+x, c:c+y]
    inp_nonzero = np.flatnonzero(inp_window)
    wt_nonzero = np.flatnonzero(weight[channel])
    common = sum(X == Y for X, Y in zip(inp_nonzero, wt_nonzero))
    mult_nonzero += common

  for tup in pos:
    conv_mult_count += 1
    img_out_cell += tup[0]*input[0][tup[1]][r+tup[2]][c+tup[3]]

  img_out_cell+=bias

  for tup in neg:
    if img_out_cell < 0:
      break
    conv_mult_count += 1
    img_out_cell += tup[0]*input[0][tup[1]][r+tup[2]][c+tup[3]]

  return img_out_cell, conv_mult_count, mult_nonzero


def compute_filter_conv_sparsepred(input, weights, comp_wt, width_out, height_out,stride=(1,1), padding=(0,0), bias=0):
#     print('called compute_filter_conv')
    img_out_channel = torch.zeros(width_out, height_out)
    filter_mult_count = 0
    filter_calc_mult = 0
    for r in range(0,input.shape[2]+2*padding[0]-weights.shape[1]+1,stride[0]):
        for c in range(0,input.shape[3]+2*padding[1]-weights.shape[2]+1,stride[1]):
            r_out = int(r/stride[0])
            c_out = int(c/stride[1])
            img_out_channel[r_out][c_out], mult_count, calc_mult = compute_conv_sparsepred(input, weights,comp_wt, r, c, bias)
            filter_mult_count += mult_count
            filter_calc_mult += calc_mult
    return img_out_channel, filter_mult_count, filter_calc_mult


def myconv2d_sparse_pred(input, weight, bias=None, stride=(1,1), padding=(0,0), dilation=(1,1), groups=1):
    input = torch.nn.functional.pad(input, (padding[1], padding[1], padding[0], padding[0]), "constant", 0)
    in_x = input.shape[2]
    in_y = input.shape[3]
    wt_x = weight.shape[2]
    wt_y = weight.shape[3]
    c = weight.shape[1]
    filter_count = weight.shape[0]
    w = int((in_x+2*padding[0]-wt_x)//stride[0]+1)
    h = int((in_y+2*padding[1]-wt_y)//stride[1]+1)
    out = torch.empty(size=(1, filter_count, w, h))

    mult_count = 0
    calc_mult = 0
    for i in range(filter_count):
        comp_wt = comp_vector_pred(weight[i])
        out[0][i], num1, num2 =compute_filter_conv_sparsepred(input, weight[i], comp_wt, w, h,stride, padding, bias[i])
        # out[0][i] += bias[i]
        mult_count+=num1
        calc_mult+=num2
    # mult_count is predictive sparse(weight only)
    # calc_mult is baseline 2(sparse non-predictive)
#     print(out)
    return (out,mult_count)
###################################################################################

# ALGO 5
class comp_vector_pred_twosided():
  def __init__(self, arr):
    self.x = arr.size(dim=2)
    self.y = arr.size(dim=1)
    self.c = arr.size(dim=0)
    self.pos_vector = [] #stores tuples of (data, index)
    self.neg_vector = []

    for k in range(self.c):
            for i in range(self.y):
                for j in range(self.x):
                    w = arr[k][i][j]
                    if w > 0:
                      self.pos_vector.append(tuple((w, k, i, j)))
                    elif w<0:
                      self.neg_vector.append(tuple((w, k, i, j)))

    self.neg_vector = sorted(self.neg_vector, key = lambda x: x[0])

  def get_pos_vector(self):
    return self.pos_vector

  def get_neg_vector(self):
    return self.neg_vector


def compute_conv_sparsepred_twosided(input, weight, comp_wt, r, c,bias=0):
  img_out_cell = 0
  conv_mult_count = 0
  pos = comp_wt.get_pos_vector()
  neg = comp_wt.get_neg_vector()

  x = weight.shape[1]
  y = weight.shape[2]
  k = weight.shape[0]

  mult_nonzero = 0
  for channel in range(k):
    inp_window = input[0][channel][r:r+x, c:c+y]
    inp_nonzero = np.flatnonzero(inp_window)
    wt_nonzero = np.flatnonzero(weight[channel])
    common = sum(X == Y for X, Y in zip(inp_nonzero, wt_nonzero))
    mult_nonzero += common

  for tup in pos:
    if(input[0][tup[1]][r+tup[2]][c+tup[3]]==0):
      continue
    conv_mult_count += 1
    img_out_cell += tup[0]*input[0][tup[1]][r+tup[2]][c+tup[3]]

  img_out_cell+=bias

  # idx = 0
  # while img_out_cell>=0 and idx<len(neg):
  #   tup = neg[idx]
  #   if(input[0][tup[1]][r+tup[2]][c+tup[3]]==0):
  #     continue
  #   conv_mult_count += 1
  #   img_out_cell += tup[0]*input[0][tup[1]][r+tup[2]][c+tup[3]]
  #   idx+=1

  for tup in neg:
    if(input[0][tup[1]][r+tup[2]][c+tup[3]]==0):
      continue
    if img_out_cell < 0:
      break
    conv_mult_count += 1
    img_out_cell += tup[0]*input[0][tup[1]][r+tup[2]][c+tup[3]]

  return img_out_cell, conv_mult_count, mult_nonzero


def compute_filter_conv_sparsepred_twosided(input, weights, comp_wt, width_out, height_out,stride=(1,1), padding=(0,0), bias=0):
    img_out_channel = torch.zeros(width_out, height_out)
    filter_mult_count = 0
    filter_calc_mult = 0
    # print(input.shape)
    # print(weights.shape)
    # print(len(stride))
    # print(len(padding))
    for r in range(0,input.shape[2]+2*padding[0]-weights.shape[1]+1,stride[0]):
        for c in range(0,input.shape[3]+2*padding[1]-weights.shape[2]+1,stride[1]):
            r_out = int(r/stride[0])
            c_out = int(c/stride[1])
            img_out_channel[r_out][c_out], mult_count, calc_mult = compute_conv_sparsepred_twosided(input, weights,comp_wt, r, c, bias)
            # img_out_channel[r][c] += bias 
            # Bias added in compute_conv_sparsepred_twosided
            filter_mult_count += mult_count
            filter_calc_mult += calc_mult
    return img_out_channel, filter_mult_count, filter_calc_mult


def myconv2d_sparse_pred_twosided(input, weight, bias=None, stride=(1,1), padding=(0,0), dilation=(1,1), groups=1):
  input = torch.nn.functional.pad(input, (padding[1], padding[1], padding[0], padding[0]), "constant", 0)
  in_x = input.shape[2]
  in_y = input.shape[3]
  wt_x = weight.shape[2]
  wt_y = weight.shape[3]
  c = weight.shape[1]
  filter_count = weight.shape[0]
  w = int((in_x+2*padding[0]-wt_x)//stride[0]+1)
  h = int((in_y+2*padding[1]-wt_y)//stride[1]+1)
  # print(w,h)
  out = torch.empty(size=(1, filter_count, w, h))

  mult_count = 0
  calc_mult = 0
  for i in range(filter_count):
    comp_wt = comp_vector_pred_twosided(weight[i])
    out[0][i], num1, num2 =compute_filter_conv_sparsepred_twosided(input, weight[i], comp_wt, w, h,stride=stride,padding=padding,bias=bias[i])
    mult_count+=num1
    calc_mult+=num2
  
  return (out,mult_count)


**Defining custom Conv2D Layer**

In [6]:
class Custom_Conv2d(torch.nn.modules.conv._ConvNd):
    """
    Implements a standard convolution layer that can be used as a regular module
    """
    def __init__(self, in_channels, out_channels, kernel_size, stride=1,
                 padding=0, dilation=1, groups=1,
                 bias=True, padding_mode='zeros'):
        kernel_size = (kernel_size, kernel_size)
        stride = (stride, stride)
        padding = (padding, padding)
        dilation = (dilation, dilation)
        super(Custom_Conv2d, self).__init__(
            in_channels, out_channels, kernel_size, stride, padding, dilation,
            False, (0, 0), groups, bias, padding_mode)

    def conv2d_forward(self, input, weight):
        return myconv2d_sparse_pred_twosided(input, weight, self.bias, self.stride,
                        self.padding, self.dilation, self.groups)

    def forward(self, input):
        return self.conv2d_forward(input, self.weight)


**Defining custom AlexNet**

In [7]:
# empty arrays for storing activation sparsities
c1 = []
c2 = []

In [8]:
# empty arrays for storing #MACops per layer
m1 = []
m2 = []

In [9]:
class CustomLeNet(nn.Module):
    def __init__(self, num_classes=10):
        super(CustomLeNet, self).__init__()
        self.features = nn.Sequential(
            Custom_Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
            Custom_Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        
        self.classifier = nn.Sequential(
            nn.Linear(400, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, num_classes)
        )
        
    def forward(self, x):
        # print('Sparsity of CONV1 activations: ', (1 - torch.count_nonzero(x)/torch.numel(x)).item())
        c1.append((1 - torch.count_nonzero(x)/torch.numel(x)).item())
#         print(x.size())
        out, macops1 = self.features[0](x) 
#         print(out.size())
        m1.append(macops1)
        movingtime = 0
        s = time.time()
        out = out.to(device)
        movingtime += time.time()-s
        out = self.features[1](out)
        out = self.features[2](out)
        out = self.features[3](out)
        
        # print('Sparsity of CONV2 activations: ', (1 - torch.count_nonzero(out)/torch.numel(out)).item())
        c2.append((1 - torch.count_nonzero(out)/torch.numel(out)).item())
#         print(out.size())
        out, macops2 = self.features[4](out)
#         print(out.size())
        m2.append(macops2)
        s = time.time()
        out = out.to(device)
        movingtime += time.time()-s
#         print('movingtime: ', movingtime)
        out = self.features[5](out)
        out = self.features[6](out)
        out = self.features[7](out)
        
        out = out.reshape(out.size(0), -1)
        out = self.classifier(out)
        return out



**Instantiating a custom LeNet**

In [10]:
cust_lenet = CustomLeNet(10)

In [11]:
# load weights
cust_lenet.load_state_dict(torch.load(r'C:\Users\rjsha\Downloads\lenet_unnormalised.pth'))
cust_lenet.to(device)
cust_lenet.eval()

CustomLeNet(
  (features): Sequential(
    (0): Custom_Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Custom_Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=400, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): ReLU()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)

# Measurements

**Accuracy & Layer-wise Activation Sparsities(Dynamic)**

In [12]:
def test_model(model):
  model.to(device)
  model.eval()
  correct = 0
  total = 0
  m1.clear()
  m2.clear()
  c1.clear()
  c2.clear()
  with torch.no_grad():
      for data in test_loader:
          if(total>=100):
            break
          images, labels = data[0].to(device), data[1].to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  print('Accuracy of the network on the test images: %.2f %%' % (100 * correct / total))
  print("CONV1 #MACops(avg):", sum(m1)/len(m1))
  print("CONV2 #MACops(avg):", sum(m2)/len(m2))

  print("CONV1 activation sparsity(avg):", sum(c1)/len(c1))
  print("CONV2 activation sparsity(avg):", sum(c2)/len(c2))
  # sum(c2)/len(c2)
  print('number of test images: ', len(c1))

In [29]:
with torch.no_grad():
      for data in test_loader:
            images1, labels = data[0].to(device), data[1].to(device)
            break        

In [202]:
# images1[0][0][10:20, 10:20]

tensor([[0.2118, 0.5843, 0.8157, 0.7373, 0.4941, 0.3216, 0.2118, 0.1843, 0.2196,
         0.2078],
        [0.5333, 0.8667, 0.7176, 0.3137, 0.1176, 0.0235, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.7098, 0.8745, 0.4745, 0.0314, 0.0000, 0.0000, 0.0000, 0.0000, 0.0196,
         0.1922],
        [0.6706, 0.7686, 0.3647, 0.0039, 0.0000, 0.0000, 0.0000, 0.0392, 0.2314,
         0.5961],
        [0.5725, 0.7686, 0.4667, 0.1059, 0.0471, 0.0667, 0.1333, 0.2784, 0.7137,
         0.9529],
        [0.3765, 0.7765, 0.8314, 0.5529, 0.3020, 0.3922, 0.7294, 0.9216, 0.9882,
         0.9961],
        [0.0235, 0.4510, 0.8431, 0.9373, 0.9333, 0.8941, 0.9216, 0.7804, 0.9098,
         0.9922],
        [0.0000, 0.0784, 0.1882, 0.2902, 0.3412, 0.1725, 0.1765, 0.2706, 0.7608,
         0.9255],
        [0.0000, 0.0000, 0.0118, 0.0431, 0.0588, 0.0000, 0.0000, 0.4275, 0.8431,
         0.8314],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6235, 0.9255,
         0.6627]])

In [174]:
# with torch.no_grad():
#         out = myconv2d(images1, cust_lenet.features[0].weight, bias=cust_lenet.features[0].bias)[0]
#         out = cust_lenet.features[1](out)
#         out = cust_lenet.features[2](out)

tensor([[[[-0.0570, -0.0570, -0.0570,  ..., -0.0570, -0.0570, -0.0570],
          [-0.0570, -0.0570, -0.0570,  ..., -0.0570, -0.0570, -0.0570],
          [-0.0570, -0.0570, -0.0570,  ..., -0.0570, -0.0570, -0.0570],
          ...,
          [-0.0570, -0.0577, -0.0594,  ..., -0.0570, -0.0570, -0.0570],
          [-0.0570, -0.0570, -0.0570,  ..., -0.0570, -0.0570, -0.0570],
          [-0.0570, -0.0570, -0.0570,  ..., -0.0570, -0.0570, -0.0570]],

         [[ 0.1148,  0.1148,  0.1148,  ...,  0.1148,  0.1148,  0.1148],
          [ 0.1148,  0.1148,  0.1148,  ...,  0.1148,  0.1148,  0.1148],
          [ 0.1148,  0.1148,  0.1148,  ...,  0.1148,  0.1148,  0.1148],
          ...,
          [ 0.1148,  0.1133,  0.1037,  ...,  0.1148,  0.1148,  0.1148],
          [ 0.1148,  0.1148,  0.1148,  ...,  0.1148,  0.1148,  0.1148],
          [ 0.1148,  0.1148,  0.1148,  ...,  0.1148,  0.1148,  0.1148]],

         [[-0.4856, -0.4856, -0.4856,  ..., -0.4856, -0.4856, -0.4856],
          [-0.4856, -0.4856, -

In [175]:
# with torch.no_grad():
#         out = myconv2d_sparse_pred(images1, cust_lenet.features[0].weight, bias=cust_lenet.features[0].bias)[0]
#         print(cust_lenet.features[1](out))

tensor([[[[-0.0570, -0.0570, -0.0570,  ..., -0.0570, -0.0570, -0.0570],
          [-0.0570, -0.0570, -0.0570,  ..., -0.0570, -0.0570, -0.0570],
          [-0.0570, -0.0570, -0.0570,  ..., -0.0570, -0.0570, -0.0570],
          ...,
          [-0.0570, -0.0577, -0.0594,  ..., -0.0570, -0.0570, -0.0570],
          [-0.0570, -0.0570, -0.0570,  ..., -0.0570, -0.0570, -0.0570],
          [-0.0570, -0.0570, -0.0570,  ..., -0.0570, -0.0570, -0.0570]],

         [[ 0.1148,  0.1148,  0.1148,  ...,  0.1148,  0.1148,  0.1148],
          [ 0.1148,  0.1148,  0.1148,  ...,  0.1148,  0.1148,  0.1148],
          [ 0.1148,  0.1148,  0.1148,  ...,  0.1148,  0.1148,  0.1148],
          ...,
          [ 0.1148,  0.1133,  0.1123,  ...,  0.1148,  0.1148,  0.1148],
          [ 0.1148,  0.1148,  0.1148,  ...,  0.1148,  0.1148,  0.1148],
          [ 0.1148,  0.1148,  0.1148,  ...,  0.1148,  0.1148,  0.1148]],

         [[-0.4856, -0.4856, -0.4856,  ..., -0.4856, -0.4856, -0.4856],
          [-0.4856, -0.4856, -

In [91]:
start = time.time()
test_model(cust_lenet)
print(time.time()-start)

kernel_id 0
kernel_id 0
kernel_id 8
kernel_id 0
kernel_id 0
kernel_id 8
kernel_id 0
kernel_id 0
kernel_id 8
kernel_id 0
kernel_id 0
kernel_id 8
kernel_id 0
kernel_id 0
kernel_id 8
kernel_id 0
kernel_id 0
kernel_id 8
kernel_id 0
kernel_id 0
kernel_id 8
kernel_id 0
kernel_id 0
kernel_id 8
kernel_id 0
kernel_id 0
kernel_id 8
kernel_id 0
kernel_id 0
kernel_id 8
kernel_id 0
kernel_id 0
kernel_id 8
kernel_id 0
kernel_id 0
kernel_id 8
kernel_id 0
kernel_id 0
kernel_id 8
kernel_id 0
kernel_id 0
kernel_id 8
kernel_id 0
kernel_id 0
kernel_id 8
kernel_id 0
kernel_id 0
kernel_id 8
kernel_id 0
kernel_id 0
kernel_id 8
kernel_id 0
kernel_id 0
kernel_id 8
kernel_id 0
kernel_id 0
kernel_id 8
kernel_id 0
kernel_id 0
kernel_id 8
kernel_id 0
kernel_id 0
kernel_id 8
kernel_id 0
kernel_id 0
kernel_id 8
kernel_id 0
kernel_id 0
kernel_id 8
kernel_id 0
kernel_id 0
kernel_id 8
kernel_id 0
kernel_id 0
kernel_id 8
kernel_id 0
kernel_id 0
kernel_id 8
kernel_id 0
kernel_id 0
kernel_id 8
kernel_id 0
kernel_id 0
kern

**Weight Sparsity (static)**

In [13]:
def get_lenet_w_sparsities(model):
  conv_indices = [0, 4]

  for i in range(2):
    layer_index = conv_indices[i]

    print(
        "Sparsity in conv{:}.weight: {:.2f}%".format(i+1, 
            100. * float(torch.sum(model.features[layer_index].weight == 0))
            / float(model.features[layer_index].weight.nelement())
        )
    )

In [14]:
get_lenet_w_sparsities(cust_lenet)

Sparsity in conv1.weight: 0.00%
Sparsity in conv2.weight: 0.00%


# Pruning

In [15]:
import torch.nn.utils.prune as prune

In [24]:
lenet_25 = CustomLeNet(10)
lenet_25.load_state_dict(torch.load(r'C:\Users\rjsha\Downloads\lenet.pth'))

for name, module in lenet_25.named_modules():
    # prune 25% of connections in all 2D-conv layers
    if isinstance(module, Custom_Conv2d):
        prune.l1_unstructured(module, name='weight', amount=0.90)

In [25]:
get_lenet_w_sparsities(lenet_25)

Sparsity in conv1.weight: 90.00%
Sparsity in conv2.weight: 90.00%


In [357]:
test_model(lenet_25)

Accuracy of the network on the test images: 9.58 %
CONV1 #MACops(avg): 117600.0
CONV2 #MACops(avg): 240000.0
CONV1 activation sparsity(avg): 0.0
CONV2 activation sparsity(avg): 0.4483547618865967
number of test images:  5000


In [16]:
model = CustomLeNet(10)
model.load_state_dict(torch.load(r'C:\Users\rjsha\Downloads\lenet_unnormalised.pth'))
model.to(device)
model.eval()

parameters_to_prune = (
    (model.features[0], 'weight'),
    (model.features[4], 'weight')
)

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.0,
)

In [17]:
get_lenet_w_sparsities(model)

Sparsity in conv1.weight: 0.00%
Sparsity in conv2.weight: 0.00%


In [18]:
start = time.time()
test_model(model)
print(time.time()-start)

Accuracy of the network on the test images: 91.00 %
CONV1 #MACops(avg): 74432.44
CONV2 #MACops(avg): 106843.69
CONV1 activation sparsity(avg): 0.0
CONV2 activation sparsity(avg): 0.6182653045654297
number of test images:  100
883.6397612094879


In [19]:
model = CustomLeNet(10)
model.load_state_dict(torch.load(r'C:\Users\rjsha\Downloads\lenet_unnormalised.pth'))
model.to(device)
model.eval()

parameters_to_prune = (
    (model.features[0], 'weight'),
    (model.features[4], 'weight')
)

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.25,
)

In [20]:
start = time.time()
test_model(model)
print(time.time()-start)

Accuracy of the network on the test images: 87.00 %
CONV1 #MACops(avg): 61475.91
CONV2 #MACops(avg): 82405.74
CONV1 activation sparsity(avg): 0.0
CONV2 activation sparsity(avg): 0.6241241455078125
number of test images:  100
885.2152101993561


In [21]:
model = CustomLeNet(10)
model.load_state_dict(torch.load(r'C:\Users\rjsha\Downloads\lenet_unnormalised.pth'))
model.to(device)
model.eval()

parameters_to_prune = (
    (model.features[0], 'weight'),
    (model.features[4], 'weight')
)

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.50,
)

In [22]:
start = time.time()
test_model(model)
print(time.time()-start)

Accuracy of the network on the test images: 87.00 %
CONV1 #MACops(avg): 50100.01
CONV2 #MACops(avg): 52344.15
CONV1 activation sparsity(avg): 0.0
CONV2 activation sparsity(avg): 0.64349489569664
number of test images:  100
570.5057721138


In [23]:
model = CustomLeNet(10)
model.load_state_dict(torch.load(r'C:\Users\rjsha\Downloads\lenet_unnormalised.pth'))
model.to(device)
model.eval()

parameters_to_prune = (
    (model.features[0], 'weight'),
    (model.features[4], 'weight')
)

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.75,
)

In [24]:
start = time.time()
test_model(model)
print(time.time()-start)

Accuracy of the network on the test images: 79.00 %
CONV1 #MACops(avg): 39107.71
CONV2 #MACops(avg): 24672.83
CONV1 activation sparsity(avg): 0.0
CONV2 activation sparsity(avg): 0.6399064642190934
number of test images:  100
365.70060563087463
